# 게시판
> 블로그에서 확장되는 개념=> 일반화하기
- 조회수, 이미지 upload -> models.py에서 해결
- 1개 게시판 -> n개
- paging / 동적 loading

# 1. setting
- python manamge.py startapp myboard

> 기존의 app에서 복사할 것
- admin.py
- forms.py
- models.py
- urls.py
- views.py

> template도 복사

In [3]:
# settings.py
'''
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'myapp',
    'django_extensions',
    'ajax',
    'blog',
    'myboard', 
]

'''

"\nINSTALLED_APPS = [\n    'django.contrib.admin',\n    'django.contrib.auth',\n    'django.contrib.contenttypes',\n    'django.contrib.sessions',\n    'django.contrib.messages',\n    'django.contrib.staticfiles',\n    'myapp',\n    'django_extensions',\n    'ajax',\n    'blog',\n    'myboard', \n]\n\n"

In [2]:
# urls.py
'''
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('ajax/', include('ajax.urls')),
    path('', include('myapp.urls')),
    path('blog/', include('blog.urls')),
    path('myboard/', include('myboard.urls')),

]


'''

"\nfrom django.contrib import admin\nfrom django.urls import path, include\n\nurlpatterns = [\n    path('admin/', admin.site.urls),\n    path('ajax/', include('ajax.urls')),\n    path('', include('myapp.urls')),\n    path('blog/', include('blog.urls')),\n    path('myboard/', include('myboard.urls')),\n\n]\n\n\n"

# 2. model 만들기

In [ ]:
# models.py
'''
from django.db import models
from django.utils import timezone

# Create your models here.
class Board(models.Model):
    
    author = models.ForeignKey('auth.User',
                            on_delete= models.CASCADE, 
                            )
                                 
    title = models.CharField(max_length = 200)
    text = models.TextField()  # textarea로 받는 부분
    created_data = models.DateTimeField(default =timezone.now)
    published_date = models.DateTimeField(blank = True, # application 관점에서 data가 비어도 되는지
                                            null = True) # 데이터 베이서 관점에서 널값 허용

    cnt = models.IntegerField(default = 0)
    image = models.CharField(max_length=200, null = True, blank = True)
    
    # QnA인지 게시판 별 구분할 수 있는 filed
    category = models.CharField(max_length = 10, default = 'common')
    
    # 프린트 함수
    def __str__(self):
        return self.title


'''

## admin.py에서 db 등록하기

In [4]:
# admin.py
'''
from django.contrib import admin
from . import models
# Register your models here.

admin.site.register(models.Board)
'''

'\nfrom django.contrib import admin\nfrom . import models\n# Register your models here.\n\nadmin.site.register(Board)\n'

# 3. Forms.py 수정

In [ ]:
# forms.py
'''

from . import models
from django.forms import ValidationError
from django import forms


def validator(value):
    if len(value) < 5 : raise  ValidationError('길이가 너무 짧아요')


class BoardForm(forms.ModelForm):
        # 부가적인 정보를 입력하는 calss
    class Meta:
        # model: 정해진 변수명
        model = models.Borad
        fields = ['title', 'text', 'category]

        # 입력할 때 category, image도 처리해야 함.-> image field는 추가로 처리하기
        
    def __init__(self, *args, **kwargs):
        super(BoardForm, self).__init__(*args, **kwargs)
        self.fields['title'].validator=[validator]


'''

# urls.py 수정

In [ ]:
'''

from . import models
from django.forms import ValidationError
from django import forms


def validator(value):
    if len(value) < 5 : raise  ValidationError('길이가 너무 짧아요')


class BoardForm(forms.ModelForm):
        # 부가적인 정보를 입력하는 calss
    class Meta:
        # model: 정해진 변수명
        model = models.Borad
        fields = ['title', 'text', 'category]

        # 입력할 때 category, image도 처리해야 함.-> image field는 추가로 처리하기
        
    def __init__(self, *args, **kwargs):
        super(BoardForm, self).__init__(*args, **kwargs)
        self.fields['title'].validator=[validator]


'''

# 4. Views.py 수정

In [ ]:
'''
from django.shortcuts import render, get_object_or_404,redirect
from django.http import HttpResponse
from django.views.generic import View
from django.contrib.auth import authenticate
from django.contrib.auth.models import User
from django.forms import CharField, Textarea, ValidationError
from django import forms
#from blog.forms import PostForm  # 프로젝트 명이 바뀌면 blog를 바꿔야하므로 좋지 않음
from . import models
from . import forms

# url: bolg -> index함수 실행
def index(request):
    return HttpResponse('ok')

### <int:pk>/ <mode>를 url을 사용하기 위해
class BoardView(View) :
    def get(self, request, pk, mode):
        print(pk, mode)
        if mode =='add':
            # pk값은 상관없음.
            form = forms.BoardForm()
            return render(request, 'myboard/edit.html', {'form': form})
        elif mode == 'list':
            # pk값 상관없음.
            username = request.session.get('username', '')
            user = User.objects.get(username =username)
            data= models.Board.objects.all().filter(author = user)
            context = {'data': data,'username': username}
            return render (request,'myboard/list.html', context)

        elif mode =='detail':
            p = get_object_or_404(models.Board, pk = pk)
            return render(request, 'myboard/detail.html', {'d': p})

        elif mode == 'edit':
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(instance=post)

        elif mode =='delete':
            post = get_object_or_404(models.Board, pk=pk)
            post.delete()
            return redirect('/0/list')
        else:
            return HttpResponse('mode를 잘 입력하세용')
        return render(request, "myboard/edit.html", {"form":form})



    def post(self, request, pk, mode):

        username = request.session["username"]
        user = User.objects.get(username=username)

        if pk == 0:
            form = forms.BoardForm(request.POST)
        else:
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            # add할 때 저자정보 넣고
            if pk == 0:
                post.author = user
            # 아닐 때는 조회수 1 증가
            else :
                post.cnt += 1
            post.save()
            return redirect("myboard", 0, 'list')  
        return render(request, "myboard/edit.html", {"form": form})
            
          



'''

# migration
- python manage.py makemigrations
- python manage.py migrate

# 5. template.py 수정

In [ ]:
# list.py
'''
<!-- import, includ 개념 -->
{% extends 'myboard/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}
    <br>
   
    <a href="{% url 'myboard' 0 'add' %}">Add </a> <br>
    여기부터 sub-template<br>

    {% for d in data %}
    
            <a href="{% url 'myboard' d.pk 'detail' %}"> {{d.title}}</a>
       <br>
    {% endfor %}


    

{% endblock %}

'''

In [ ]:
# detail.py
'''
<!-- import, includ 개념 -->
{% extends 'myboard/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

게시물을 보여줘<br>

{{ form.as_p }}
   

    <a href="{%  url 'myboard' d.pk 'edit'  %}">수정</a>

{% endblock %}

'''

In [ ]:
# edit.py
'''

<script src="http://code.jquery.com/jquery-1.11.3.min.js"></script>
<script src="http://code.jquery.com/jquery-migrate-1.2.1.min.js"></script>

 <style>
      .bg { background-color: #eeeeee; }
      .bd { border: 1px solid #666666; }
 </style>

{% if form.title.value %}
<h1>  수정하기 </h1>
{% else %}
<h1>  신규작성</h1>
{% endif %}


<form method=post>
   {% csrf_token %}
   	{{ form.as_p }}

   <input type="submit" value="작성" >

</form>

<script>
    // table에서 id의 title 노드에 bg, bd 속성 적용하기
   $("#id_title").addClass('bg bd');
</script>


'''

In [ ]:
# base.html
'''
<h2>게시판 만들기</h2><br>

사용자: {{username}}


{% block content %}
{% endblock %}



<br>
<br>
<br>

copy right.....<br>
서울 특별시
'''